## Imports

In [1]:
# DO NOT CHANGE THESE LINES.
import os
import json
import pandas as pd
import warnings
from sklearn.linear_model import LogisticRegression
from feature_engine.encoding import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from joblib import dump
warnings.filterwarnings('ignore')

## Paths

In [2]:
# DO NOT CHANGE THESE LINES.
ROOT_DIR = os.path.dirname(os.getcwd())
MODEL_INPUTS_OUTPUTS = os.path.join(ROOT_DIR, 'model_inputs_outputs/')
INPUT_DIR = os.path.join(MODEL_INPUTS_OUTPUTS, "inputs")
INPUT_SCHEMA_DIR = os.path.join(INPUT_DIR, "schema")
DATA_DIR = os.path.join(INPUT_DIR, "data")
TRAIN_DIR = os.path.join(DATA_DIR, "training")
TEST_DIR = os.path.join(DATA_DIR, "testing")
MODEL_PATH = os.path.join(MODEL_INPUTS_OUTPUTS, "model")
MODEL_ARTIFACTS_PATH = os.path.join(MODEL_PATH, "artifacts")
OHE_ENCODER_FILE = os.path.join(MODEL_ARTIFACTS_PATH, 'ohe.joblib')
PREDICTOR_DIR_PATH = os.path.join(MODEL_ARTIFACTS_PATH, "predictor")
PREDICTOR_FILE_PATH = os.path.join(PREDICTOR_DIR_PATH, "predictor.joblib")
IMPUTATION_FILE = os.path.join(MODEL_ARTIFACTS_PATH, 'imputation.joblib')
LABEL_ENCODER_FILE = os.path.join(MODEL_ARTIFACTS_PATH, 'label_encoder.joblib')
if not os.path.exists(MODEL_ARTIFACTS_PATH):
    os.makedirs(MODEL_ARTIFACTS_PATH)
if not os.path.exists(PREDICTOR_DIR_PATH):
    os.makedirs(PREDICTOR_DIR_PATH)

### Reading the schema
The schema contains metadata about the datasets. We will use the scehma to get information about the type of each feature (NUMERIC or CATEGORICAL) and the id and target features, this will be helpful in preprocessing stage.

In [3]:
file_name = [f for f in os.listdir(INPUT_SCHEMA_DIR) if f.endswith('json')][0]
schema_path = os.path.join(INPUT_SCHEMA_DIR, file_name)
with open(schema_path, "r", encoding="utf-8") as file:
    schema = json.load(file)
features = schema['features']

numeric_features = []
categorical_features = []
for f in features:
    if f['dataType'] == 'CATEGORICAL':
        categorical_features.append(f['name'])
    else:
        numeric_features.append(f['name'])

id_feature = schema['id']['name']
target_feature = schema['target']['name']
target_classes = schema['target']['classes']

### Reading training data

In [4]:
file_name = [f for f in os.listdir(TRAIN_DIR) if f.endswith('.csv')][0]
file_path = os.path.join(TRAIN_DIR, file_name)
df = pd.read_csv(file_path)
df.head()

,Id,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X24,X25,X26,X27,X28,X29,X30,X31,X32,Phase
0,5843,0.008641,-0.002755,-0.001820,0.000991,0.000656,0.000014,0.008141,-0.002648,-0.001606,...,-0.000009,0.009250,0.001189,0.008710,0.003422,0.000674,0.000038,0.000342,0.000192,H
1,1776,-0.000973,0.001626,-0.000072,-0.000420,0.003060,-0.000281,0.000463,0.001564,-0.000300,...,-0.000005,0.001897,0.003102,0.001659,0.003012,0.000317,0.002450,0.000303,0.000131,P
2,5522,0.006124,-0.030467,0.002402,0.004168,-0.000922,-0.000280,0.003658,-0.021807,0.001822,...,-0.000001,0.031169,0.004278,0.022187,0.002447,0.000973,0.000101,0.001329,0.000047,P
3,7047,0.010721,0.005850,0.031741,0.019252,0.017786,0.060170,0.010197,0.002923,0.010932,...,0.003263,0.034010,0.065631,0.015233,0.039471,0.002025,0.004293,0.000824,0.003670,S
4,7152,0.006200,0.015449,-0.008240,-0.000140,-0.001091,-0.001790,0.004247,0.008129,-0.006152,...,-0.001890,0.018574,0.002101,0.011043,0.001375,0.002605,0.001816,0.000537,0.001984,S


## Data Preprocessing
Data preprocessing is very important before training the model, as the data may contain missing values in some cells. Moreover, most of the learning algorithms cannot work with categorical data, thus the data has to be encoded.

In this section we will impute the missing values and encode the categorical features. Afterwards the data will be ready to train the model.

##### Imputing missing data
> The median value will be used to impute missing values of the numeric features and the mode will be used to impute categorical features.

##### You can add your own preprocessing steps such as:
<ul>
<li>Normalization</li> <br>
<li>Outlier removal</li><br>
<li>Handling imbalanced classes</li><br>
<li>Dropping or adding features</li><br>
</ul>

### Important note:
<p> 
Saving the values used for imputation during training step is crucial. These values will be used to impute missing data in the testing set. This is very important to avoid the well known problem of data leakage. During testing, you should not make any assumptions about the data in hand, alternatively anything needed during the testing phase should be learned from the training phase. This is why we are creating a dictionary of values used during training to reuse these values during testing.
</p>


In [5]:
# Imputing missing data
imputation_values = {}
columns_with_missing_values = df.columns[df.isna().any()]
for column in columns_with_missing_values:    
    if column in numeric_features:
        value = df[column].median()
    else:
        value = df[column].mode()[0]
    df[column].fillna(value, inplace=True)
    imputation_values[column] = value

dump(imputation_values, IMPUTATION_FILE)

# Comment the above code and write you own imputation code here

"""
BEGIN

CODE HERE

END
"""


'\nBEGIN\n\nCODE HERE\n\nEND\n'

##### Encoding Categorical features
<p>
The id column is just an identifier for the training example, so we will exclude it during the encoding phase.<br>
Target feature will be label encoded in the next step.
</p>


In [6]:
# Saving the id and target columns in a different variable.
ids = df[id_feature]
target = df[target_feature]

# Dropping the id and target from the dataframe
df.drop(columns=[id_feature, target_feature], inplace=True)

# Ensure that all categorical columns are stored as str type.
# This is to ensure that even if the categories are numbers (1, 2, ...), they still get encoded.
for c in categorical_features:
    df[c] = df[c].astype(str)

# Encoding the categorical features if exist
if categorical_features:
    encoder = OneHotEncoder(top_categories=6)
    encoder.fit(df)
    df = encoder.transform(df)

    # Saving the encoder to use it on the testing dataset
    path = dump(encoder, OHE_ENCODER_FILE)

#### Encoding the target feature

In [7]:
encoder = LabelEncoder()
y = encoder.fit_transform(target.values.reshape(-1, 1))
dump(encoder, LABEL_ENCODER_FILE)
y

array([1, 2, 2, ..., 4, 3, 2])

### Training the Classifier
We choose Logistic Regression Classifier, but feel free to try your own and compare the results.

In [8]:
# Creating a logistic regression model and training it
model = LogisticRegression()
model.fit(df, y)

"""
BEGIN

model = ...

END
"""

# Saving the model to use it for predictions
path = dump(model, PREDICTOR_FILE_PATH)
